<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/Simple_transfer_learning_with_image_modules_from_TensorFlow_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://github.com/tensorflow/hub/tree/master/examples/image_retraining


Простое обучение переносу с помощью модулей изображений из TensorFlow Hub.
В этом примере показано, как обучить классификатор изображений на основе любого
модуля TensorFlow Hub, который вычисляет векторы элементов изображения. По умолчанию,
он использует векторы функций, вычисленные Inception V3, обученные в ImageNet.
См. Https://github.com/tensorflow/hub/blob/master/docs/modules/image.md.

Верхний слой получает в качестве входных данных 2048-мерный вектор (при условии
Inception V3) для каждого изображения. Мы тренируем слой softmax поверх этого
представления. Если слой softmax содержит N меток, это соответствует 
изучению N + 2048 * N параметров модели для смещений и весов.
Вот пример, который предполагает, что у вас есть папка с именем класса, каждая из которых
содержит изображения для каждой метки. Папка с примерами flower_photosдолжна иметь такую структуру:
~ / Flower_photos / ромашка / photo1.jpg
~ / Flower_photos / ромашка / photo2.jpg
...
~ / Flower_photos / роза / anotherphoto77.jpg
...
~ / Flower_photos / Подсолнечник / somepicture.jpg

Имена подпапок важны, так как они определяют, к какой метке применяется
каждое изображение, но сами имена файлов не имеют значения. (Для рабочего примера
скачать http://download.tensorflow.org/example_images/flower_photos.tgz
и запустите tar xzf flower_photos.tgz, чтобы распаковать его.)
Как только ваши изображения подготовлены, и у вас есть установленный Тензор-хаб и достаточно свежая версия tensenflow, вы можете запустить обучение с команды как эта:
```bash
python retrain.py --image_dir ~ / flower_photos
```
Вы можете заменить аргумент image_dir любой папкой, содержащей подпапки изображений. Метка для каждого изображения берется из названия подпапки.
Это создает новый файл модели, который может быть загружен и запущен любым TensorFlow запрограммируйте, например, пример кода 
tenorflow / examples / label_image. По умолчанию этот скрипт будет использовать очень точный, но сравнительно большой и медленная Inception V3 
модель архитектуры. Рекомендуется начать с этого чтобы убедиться, что вы собрали хорошие учебные данные, но если вы хотите развернуть
на платформах с ограниченными ресурсами вы можете попробовать флаг `--tfhub_module` с модели mobilenet. Для получения дополнительной информации о 
Mobilenet, см. https://research.googleblog.com/2017/06/mobilenets-open-source-models-for.html
Например:
Запустите версию Mobilenet с плавающей запятой:
`` `Баш
python retrain.py --image_dir ~ / flower_photos \
    --tfhub_module https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/feature_vector/1
`` `
Запустить Mobilenet, инструментированный для квантования:
`` `Баш
python retrain.py --image_dir ~ / flower_photos / \
    --tfhub_module https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/quantops/feature_vector/1
`` `
Эти инструментальные модели могут быть преобразованы в полностью квантованные мобильные модели с помощью
TensorFlow Lite.
На выбор предлагаются разные модели Mobilenet с различными файлами.
параметры размера и задержки.
  - Первое число может быть «100», «075», «050» или «025» для управления номером.
    нейронов (активация скрытых слоев); количество весов (и, следовательно,
    в некоторой степени размер файла и скорость) уменьшается с квадратом этого
    доля.
  - Второе число - размер входного изображения. Вы можете выбрать «224», «192»,
    «160» или «128» с меньшими размерами, обеспечивающими более высокие скорости.
Для использования с TensorBoard:
По умолчанию этот скрипт будет записывать сводки в каталог / tmp / retrain_logs
Визуализируйте итоги с помощью этой команды:
тензорная доска --logdir / tmp / retrain_logs
Чтобы использовать с Tensorflow Serving, запустите этот инструмент с набором --saved_model_dir
в какое-либо все более пронумерованное место экспорта под базовым путем модели, например:
`` `Баш
python retrain.py (... другие аргументы, как и раньше ...) \
    --saved_model_dir = / tmp / save_models / $ (дата +% s) /
tenorflow_model_server --port = 9000 --model_name = my_image_classifier \
    --model_base_path = / TMP / saved_models /

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
from datetime import datetime
import hashlib
import os.path
import random
import re
import sys

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

FLAGS = None

MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M

# The location where variable checkpoints will be stored.
# Место, где будут храниться переменные контрольные точки.
CHECKPOINT_NAME = '/tmp/_retrain_checkpoint'

# Модуль понимается как инструмент для квантования с TF-Lite если он содержит какие-либо из этих операций.

FAKE_QUANT_OPS = ('FakeQuantWithMinMaxVars',
                  'FakeQuantWithMinMaxVarsPerChannel')

# **create_image_lists(happy)**
Создает список тренировочных образов из файловой системы.
   Анализирует вложенные папки в каталоге изображений, разбивает их на стабильные
   обучающие, тестирующие и проверочные наборы и возвращающие структуру данных
   описание списков изображений для каждой метки и их пути.
 > ** Args:**
   *  *image_dir:* строковый путь к папке, содержащей подпапки изображений.
   * *testing_percentage: *целочисленный процент изображений, зарезервированных для тестов.
   * *validation_percentage: *целочисленный процент изображений, зарезервированных для проверки.

> **  Return:**
   * OrderedDict, содержащий запись для каждой подпапки метки с изображениями
     разделить на наборы обучения, тестирования и проверки на каждом ярлыке.
     Порядок предметов определяет индексы классов.


> ссылки:
*  [collections.OrderedDict()](https://pythonworld.ru/moduli/modul-collections.html)
* [set](https://pythonworld.ru/tipy-dannyx-v-python/mnozhestva-set-i-frozenset.html)
* [tf.gfile.Walk](https://www.tensorflow.org/api_docs/python/tf/gfile/Walk)
* [tf.gfile.Glob ](https://www.tensorflow.org/api_docs/python/tf/gfile/Glob)
* [tf.gfile.Exists](https://www.tensorflow.org/api_docs/python/tf/gfile/Exists)
* [os.path.normcase, os.path.basename](https://docs.python.org/2/library/os.path.html)
* [re.sub](https://tproger.ru/translations/regular-expression-python/)
* [str.lower](http://pythonz.net/references/named/str.lower/)
* [hashlib.sha1](https://docs.python.org/3/library/hashlib.html)

In [0]:
def create_image_lists(image_dir, testing_percentage, validation_percentage):
 
  if not tf.gfile.Exists(image_dir):   #tf.gfile.Exists Определяет, существует ли путь или нет.Истинно, если путь существует, будь то файл или каталог.
    tf.logging.error("Image directory '" + image_dir + "' not found.")
    return None
  result = collections.OrderedDict() 
  #похожий на словарь объект, но он помнит порядок, в котором ему были даны ключи.
  
  
  sub_dirs = sorted(x[0] for x in tf.gfile.Walk(image_dir)) 
  # tf.gfile.Walk Рекурсивный генератор дерева каталогов для каталогов.
  # The root directory comes first, so skip it.
  # Корневой каталог стоит первым, поэтому пропустите его
  is_root_dir = True       #?????
  for sub_dir in sub_dirs: #?????
    if is_root_dir:        #?????
      is_root_dir = False  #?????
      continue             #?????
    extensions = sorted(set(os.path.normcase(ext) for ext in ['JPEG', 'JPG', 'jpeg', 'jpg'])) 
    # аналогично просто написать ['JPEG', 'JPG', 'jpeg', 'jpg']
    # os.path.normcase(path) - нормализует регистр пути
    # set создает множество удаляет повторения 
    file_list = []
    dir_name = os.path.basename(sub_dir) #os.path.basename(path) - базовое имя пути (эквивалентно os.path.split(path)[1]).
    if dir_name == image_dir:
      continue   #Оператор continue начинает следующий проход цикла, минуя оставшееся тело цикла (for или while)
    tf.logging.info("Looking for images in '" + dir_name + "'")
    for extension in extensions:
      file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
      file_list.extend(tf.gfile.Glob(file_glob))   #  tf.gfile.Glob Возвращает список файлов, которые соответствуют заданному шаблону
    if not file_list:
      tf.logging.warning('No files found')
      continue
    if len(file_list) < 20:
      tf.logging.warning(
          'WARNING: Folder has less than 20 images, which may cause issues.')
    elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
      tf.logging.warning(
          'WARNING: Folder {} has more than {} images. Some images will '
          'never be selected.'.format(dir_name, MAX_NUM_IMAGES_PER_CLASS))
    label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())  #re.sub ищет шаблон в строке и заменяет его на указанную подстроку.
                                                                #str.lower возвращает копию исходной строки с символами приведёнными к нижнему регистру.
    training_images = []
    testing_images = []
    validation_images = []
    for file_name in file_list:
      base_name = os.path.basename(file_name)
      # Мы хотим игнорировать что-либо после '_nohash_' в имени файла, когда
       # решая, в какой набор поместить изображение, создатель набора данных может
       # группировка фотографий, которые являются близкими вариациями друг друга. Например
       # это используется в наборе данных о заболеваниях растений для группировки нескольких изображений
       # тот же лист.
      hash_name = re.sub(r'_nohash_.*$', '', file_name)
      # Это выглядит немного волшебно, но мы должны решить, должен ли этот файл
       # перейти к обучающим, тестовым или проверочным наборам, и мы хотим сохранить
       # существующие файлы в том же наборе, даже если впоследствии будет больше файлов
       # добавлено.
       # Для этого нам нужен стабильный способ решения, основанный только на имени файла
       # сам, поэтому мы делаем это хеш, а затем используем его для генерации
       # значение вероятности, которое мы используем для его назначения.
      hash_name_hashed = hashlib.sha1(tf.compat.as_bytes(hash_name)).hexdigest() 
       #hashlib.sha1().hexdigest() хеширует аргумент и возвращает захе-е значение
       # tf.compat.as_bytes Converts either bytes or unicode to bytes, using utf-8 encoding for text.
      percentage_hash = ((int(hash_name_hashed, 16) %
                          (MAX_NUM_IMAGES_PER_CLASS + 1)) *
                         (100.0 / MAX_NUM_IMAGES_PER_CLASS))
      if percentage_hash < validation_percentage:
        validation_images.append(base_name)
      elif percentage_hash < (testing_percentage + validation_percentage):
        testing_images.append(base_name)
      else:
        training_images.append(base_name)
    result[label_name] = {
        'dir': dir_name,
        'training': training_images,
        'testing': testing_images,
        'validation': validation_images,
    }
  return result

# **get_image_path()**
Возвращает путь к изображению для метки по указанному индексу.
> **  Args:**
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *label_name: *строка метки, для которой мы хотим получить изображение.
* *index:* Int смещение изображения, которое мы хотим. Это будет модулировано
     Доступно количество изображений для метки, поэтому оно может быть сколь угодно большим.
* *image_dir:* строка корневых папок подпапок, содержащих обучение
     изображений.
* *category:* Имя строки набора для извлечения изображений - обучение, тестирование или проверки.
 
 > ** Return**
* Строка пути файловой системы к изображению, которое соответствует запрошенным параметрам.


In [0]:
def get_image_path(image_lists, label_name, index, image_dir, category):
  
  if label_name not in image_lists:
    tf.logging.fatal('Label does not exist %s.', label_name)
  label_lists = image_lists[label_name]
  if category not in label_lists:
    tf.logging.fatal('Category does not exist %s.', category)
  category_list = label_lists[category]
  if not category_list:
    tf.logging.fatal('Label %s has no images in the category %s.',
                     label_name, category)
  mod_index = index % len(category_list)
  base_name = category_list[mod_index]
  sub_dir = label_lists['dir']
  full_path = os.path.join(image_dir, sub_dir, base_name)
  return full_path

# **get_bottleneck_path()**
Возвращает путь к файлу узкого места для метки по указанному индексу.
>  **Args:**
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *label_name: *строка метки, для которой мы хотим получить изображение.
* *index: *целочисленное смещение изображения, которое мы хотим. Это будет модулировано
     Доступно количество изображений для метки, поэтому оно может быть сколь угодно большим.
* *bottleneck_dir: *строка папки, содержащая кэшированные файлы значений узких мест.
* *category: *Имя строки набора для извлечения изображений - обучение, тестирование или Проверка.
* *module_name: *имя используемого модуля изображения.

> **Return:**
* Строка пути файловой системы к изображению, которое соответствует запрошенным параметрам.

> ссылки
* [string.replace](http://pythonz.net/references/named/string.replace/)


In [0]:
def get_bottleneck_path(image_lists, label_name, index, bottleneck_dir,
                        category, module_name):
  module_name = (module_name.replace('://', '~')  # .replace Возвращает копию строки, в которой заменены все вхождения указанной строки указанным значением.
                 .replace('/', '~')  # URL and Unix paths.
                 .replace(':', '~').replace('\\', '~'))  # Windows paths.
  return get_image_path(image_lists, label_name, index, bottleneck_dir,
                        category) + '_' + module_name + '.txt'

# **create_module_graph()**
 Создает график и загружает в него модуль-концентратор.
 >**Args:**
* *module_spec:* hub.ModuleSpec для используемого модуля изображения.

>**Return:**
* *TF.Graph*, который был создан.
* *bottleneck_tensor*: значения узких мест, выводимые модулем.
* *resized_input_tensor:* входные изображения, размеры которых изменяются в соответствии с ожиданиями модуля.
* *want_quanization:* логическое значение, был ли модуль оснащен с поддельной операцией квантования.

> ссылки
* [hub.get_expected_image_size](https://www.tensorflow.org/hub/api_docs/python/hub/get_expected_image_size)
* [hub.Module](https://www.tensorflow.org/hub/api_docs/python/hub/Module)
* [any](https://pythonworld.ru/osnovy/vstroennye-funkcii.html)
* [op&node](https://www.tensorflow.org/guide/extend/model_files)
* [FAKE_QUANT_OPS](https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/fake-quant-with-min-max-vars-per-channel)

In [0]:
def create_module_graph(module_spec):
  
  height, width = hub.get_expected_image_size(module_spec)  #Возвращает ожидаемые [высота, ширина] размеры входного изображения.
  with tf.Graph().as_default() as graph:
    resized_input_tensor = tf.placeholder(tf.float32, [None, height, width, 3])
    m = hub.Module(module_spec)  
    # hub.Module Модуль представляет собой часть графа TensorFlow, который можно экспортировать на диск (в соответствии с форматом SavedModel), а
    # затем повторно загрузить. 
    bottleneck_tensor = m(resized_input_tensor)
    wants_quantization = any(node.op in FAKE_QUANT_OPS
                             for node in graph.as_graph_def().node) 
    # Квантование — разбиение диапазона значений некоторой величины на конечное число уровней и округление этих значений до ближайших к ним уровней.
    # op node операция и узел 
    # any(последовательность) - Возвращает True, если хотя бы один элемент - истина. Для пустой последовательности возвращает False
  return graph, bottleneck_tensor, resized_input_tensor, wants_quantization

# **run_bottleneck_on_image()**
  Запускает вывод для изображения, чтобы извлечь сводный слой «bottleneck».
>**Args:**
* sess: Текущий активный сеанс TensorFlow.
* image_data: строка необработанных данных JPEG.
* image_data_tensor: входной слой данных на графике.
* decoded_image_tensor: Вывод исходного изменения размера изображения и предварительной обработки.
* resized_input_tensor: входной узел графа распознавания.
* bottleneck_tensor: слой перед окончательным softmax.

>**Return:**
* Numpy массив значений узкого места.

> ссылки
* [numpy.squeeze](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.squeeze.html)

In [0]:
def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            decoded_image_tensor, resized_input_tensor,
                            bottleneck_tensor):

  # First decode the JPEG image, resize it, and rescale the pixel values.
  # Сначала декодируйте изображение JPEG, измените его размер и измените размеры пикселей.
  resized_input_values = sess.run(decoded_image_tensor,
                                  {image_data_tensor: image_data})
  # Then run it through the recognition network.
  # Затем запустите его через сеть распознавания.
  bottleneck_values = sess.run(bottleneck_tensor,
                               {resized_input_tensor: resized_input_values})
  bottleneck_values = np.squeeze(bottleneck_values) # np.squeeze Удалите одномерные записи из формы массива.
  return bottleneck_values

In [0]:
def ensure_dir_exists(dir_name):
 """ 
  Убедитесь, что папка существует на диске.
   Args:
  dir_name: строка пути к папке, которую мы хотим создать.
  """
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)

# **create_bottleneck_file()**
Create a single bottleneck file

[bottleneck(oreilly)](https://learning.oreilly.com/library/view/deep-learning-with/9781786469786/9444e43d-d496-4a21-a7b8-7df767c527d6.xhtml) это термин, который TensorFlow использует для ссылки на слой непосредственно перед последним слоем, который фактически отвечает за классификацию. Таким образом, любое изображение в обучающем наборе используется несколько раз во время обучения, и вычисление слоев за узким местом занимает много времени для каждого изображения. Таким образом, кэшируя выходные данные этих нижних уровней на диске, мы избегаем тратить значительное количество времени. По умолчанию узкое место хранится в /tmp/bottleneckкаталоге.
 [bottleneck](https://ai.stackexchange.com/questions/4864/what-is-the-concept-of-tensorflow-bottlenecks)
 
 > ссылки:
 * [tf.gfile.FastGFile](https://www.tensorflow.org/api_docs/python/tf/gfile/FastGFile)

In [0]:
def create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor):
  """Create a single bottleneck file.Создайте один файл узкого места."""
  tf.logging.info('Creating bottleneck at ' + bottleneck_path)
  image_path = get_image_path(image_lists, label_name, index,
                              image_dir, category)
  if not tf.gfile.Exists(image_path):
    tf.logging.fatal('File does not exist %s', image_path)
  image_data = tf.gfile.FastGFile(image_path, 'rb').read()  #файловый ввод / вывод
  try:
    bottleneck_values = run_bottleneck_on_image(
        sess, image_data, jpeg_data_tensor, decoded_image_tensor,
        resized_input_tensor, bottleneck_tensor)
  except Exception as e:
    raise RuntimeError('Error during processing file %s (%s)' % (image_path,
                                                                 str(e)))
  bottleneck_string = ','.join(str(x) for x in bottleneck_values)
  with open(bottleneck_path, 'w') as bottleneck_file:
    bottleneck_file.write(bottleneck_string)

#** get_or_create_bottleneck(): **
Извлекает или вычисляет значения узкого места для изображения.
  Если кэшированная версия данных о узких местах существует на диске, верните ее,
  в противном случае рассчитайте данные и сохраните их на диск для дальнейшего использования.
>**Args:**
* *sess:* текущий активный сеанс TensorFlow.
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *label_name:* строка метки, для которой мы хотим получить изображение.
* *index:* целочисленное смещение изображения, которое мы хотим. Это будет по модулю Доступно количество изображений для метки, поэтому оно может быть сколь угодно большим.
* *image_dir:* строка корневых папок подпапок, содержащих обучение изображений        
* *category:* Строка имени, из которой нужно извлекать изображения - обучение, тестирование или проверка.
* *bottleneck_dir: * строка папки, содержащая кэшированные файлы значений узких мест.
* *jpeg_data_tensor:* тензор для подачи загруженных данных JPEG в.
** decoded_image_tensor:* вывод декодирования и изменения размера изображения.
* *resized_input_tensor:* входной узел графа распознавания.
* *bottleneck_tensor:* выходной тензор для значений узких мест.
* *module_name:* имя используемого модуля изображения.

>**Return:**
* Numpy массив значений, созданный слоем узкого места для изображения.


In [0]:
def get_or_create_bottleneck(sess, image_lists, label_name, index, image_dir,
                             category, bottleneck_dir, jpeg_data_tensor,
                             decoded_image_tensor, resized_input_tensor,
                             bottleneck_tensor, module_name):
 
  label_lists = image_lists[label_name]
  sub_dir = label_lists['dir']
  sub_dir_path = os.path.join(bottleneck_dir, sub_dir)
  ensure_dir_exists(sub_dir_path)
  bottleneck_path = get_bottleneck_path(image_lists, label_name, index,
                                        bottleneck_dir, category, module_name)
  if not os.path.exists(bottleneck_path):
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
  with open(bottleneck_path, 'r') as bottleneck_file:
    bottleneck_string = bottleneck_file.read()
  did_hit_error = False
  try:
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  except ValueError:
    tf.logging.warning('Invalid float found, recreating bottleneck')
    did_hit_error = True
  if did_hit_error:
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
    with open(bottleneck_path, 'r') as bottleneck_file:
      bottleneck_string = bottleneck_file.read()
# Разрешить распространение исключений здесь, так как они не должны произойти после
# свежее творение
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  return bottleneck_values

# **cache_bottlenecks()**

Гарантирует, что все узкие места обучения, тестирования и проверки кэшируются.
Потому что мы можем прочитать одно и то же изображение несколько раз (если нет искажения, применяемые во время тренировок), если мы рассчитать значения слоя узкого места один раз для каждого изображения во время предварительной обработки, а затем просто несколько раз прочитать эти кэшированные значения во время повышение квалификации. Здесь мы рассмотрим все изображения, которые мы нашли, рассчитать те, значения и сохранить их.
>**Args**:
* *sess:* текущий активный сеанс TensorFlow.
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *image_dir:* строка корневых папок подпапок, содержащих обучение
    изображений.
* *bottleneck_dir:* строка папки, содержащая кэшированные файлы значений узких мест.
* *jpeg_data_tensor:* входной тензор для данных JPEG из файла.
* *decoded_image_tensor:* вывод декодирования и изменения размера изображения.
* *resized_input_tensor:* входной узел графа распознавания.
* *bottleneck_tensor:* предпоследний выходной слой графика.
* *module_name* имя используемого модуля изображения.

>**Return**
* Ничего такого.

In [0]:
def cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir,
                      jpeg_data_tensor, decoded_image_tensor,
                      resized_input_tensor, bottleneck_tensor, module_name):

  how_many_bottlenecks = 0
  ensure_dir_exists(bottleneck_dir)
  for label_name, label_lists in image_lists.items():
    for category in ['training', 'testing', 'validation']:
      category_list = label_lists[category]
      for index, unused_base_name in enumerate(category_list):
        get_or_create_bottleneck(
            sess, image_lists, label_name, index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, module_name)

        how_many_bottlenecks += 1
        if how_many_bottlenecks % 100 == 0:
          tf.logging.info(
              str(how_many_bottlenecks) + ' bottleneck files created.')


# **def get_random_cached_bottlenecks()**
Получает значения узких мест для кэшированных изображений. Если искажения не применяются, эта функция может извлечь кэшированный узкие места значения непосредственно с диска для изображений. Он выбирает случайный набор изображения из указанной категории.
> **Args:**
* *sess:* Текущая сессия TensorFlow.
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *how_many:*  Если положительный, будет выбрана случайная выборка такого размера.
    Если отрицательный, все узкие места будут восстановлены.
    Категория: Строка имени, из которой нужно извлечь - обучение, тестирование или проверка.
* *bottleneck_dir:* строка папки, содержащая кэшированные файлы значений узких мест.
* *image_dir: *строка корневых папок подпапок, содержащих обучение изображений.
* *jpeg_data_tensor:* слой для подачи данных изображения jpeg.
* *decoded_image_tensor:* вывод декодирования и изменения размера изображения.
* *resized_input_tensor:* входной узел графа распознавания.
* *bottleneck_tensor:* выходной слой узкого места графа CNN.
* *module_name:* имя используемого модуля изображения.

> **Return**
* Список массивов узких мест, их соответствующие основные истины и
    соответствующие имена файлов.
    
> Ссылки
* [random.randrange](https://docs.python.org/3.1/library/random.html)

In [0]:
def get_random_cached_bottlenecks(sess, image_lists, how_many, category,
                                  bottleneck_dir, image_dir, jpeg_data_tensor,
                                  decoded_image_tensor, resized_input_tensor,
                                  bottleneck_tensor, module_name):

  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  filenames = []
  if how_many >= 0:
    # Retrieve a random sample of bottlenecks.
    # Получить случайную выборку узких мест.
    for unused_i in range(how_many):
      label_index = random.randrange(class_count) # random.randrange Вернуть случайно выбранный элемент из диапазона (начало, остановка, шаг) 
      label_name = list(image_lists.keys())[label_index]
      image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
      image_name = get_image_path(image_lists, label_name, image_index,
                                  image_dir, category)
      bottleneck = get_or_create_bottleneck(
          sess, image_lists, label_name, image_index, image_dir, category,
          bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
          resized_input_tensor, bottleneck_tensor, module_name)
      bottlenecks.append(bottleneck)
      ground_truths.append(label_index)
      filenames.append(image_name)
  else:
    # Retrieve all bottlenecks.
    # Получить все узкие места.
    for label_index, label_name in enumerate(image_lists.keys()):
      for image_index, image_name in enumerate(
          image_lists[label_name][category]):
        image_name = get_image_path(image_lists, label_name, image_index,
                                    image_dir, category)
        bottleneck = get_or_create_bottleneck(
            sess, image_lists, label_name, image_index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, module_name)
        bottlenecks.append(bottleneck)
        ground_truths.append(label_index)
        filenames.append(image_name)
  return bottlenecks, ground_truths, filenames


# **def get_random_distorted_bottlenecks()**
Извлекает узкие места для тренировочных образов после искажений.
Если мы тренируемся с искажениями, такими как crops, scales, или flips, мы должны
  пересчитать полную модель для каждого изображения, и мы не можем использовать кэшированные
  значения узкого места. Вместо этого мы находим случайные изображения для запрошенной категории,
  пропустите их через граф искажений, а затем полный граф, чтобы получить
  узкое место результаты для каждого.
>**Args:**
* *sess:* Текущая сессия TensorFlow.
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *how_many:* целое число возвращаемых значений узкого места.
* *category: * Строка имени, набор изображений для извлечения - обучение, тестирование,
    или проверка.
* *image_dir:* строка корневых папок подпапок, содержащих обучение
    изображений.
* *input_jpeg_tensor:* входной слой, к которому мы подаем данные изображения.
* *distorted_image:* выходной узел графика искажения.
* *resized_input_tensor:* входной узел графа распознавания.
* *bottleneck_tensor:* выходной слой узкого места графа CNN.

>**Return**:
* Список массивов узких мест и соответствующие им основные истины.
 

In [0]:
def get_random_distorted_bottlenecks(
    sess, image_lists, how_many, category, image_dir, input_jpeg_tensor,
    distorted_image, resized_input_tensor, bottleneck_tensor):

  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  for unused_i in range(how_many):
    label_index = random.randrange(class_count)
    label_name = list(image_lists.keys())[label_index]
    image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
    image_path = get_image_path(image_lists, label_name, image_index, image_dir,
                                category)
    if not tf.gfile.Exists(image_path):
      tf.logging.fatal('File does not exist %s', image_path)
    jpeg_data = tf.gfile.FastGFile(image_path, 'rb').read()
    # Обратите внимание, что мы материализовали distorted_image_data как массив numpy до
     # отправка рабочего вывода на изображение. Это включает в себя 2 копии памяти и
     # может быть оптимизирован в других реализациях.
    distorted_image_data = sess.run(distorted_image,
                                    {input_jpeg_tensor: jpeg_data})
    bottleneck_values = sess.run(bottleneck_tensor,
                                 {resized_input_tensor: distorted_image_data})
    bottleneck_values = np.squeeze(bottleneck_values)
    bottlenecks.append(bottleneck_values)
    ground_truths.append(label_index)
  return bottlenecks, ground_truths


# **def should_distort_images()**
Включены ли какие-либо искажения из входных флагов.
> **Args:**
* *flip_left_right:* Boolean, следует ли случайным образом отражать изображения по горизонтали.
* *random_crop:* целочисленный процент, задающий общую маржу, используемую вокруг ящик для обрезки.
* *random_scale:* целое число в процентах от того, насколько сильно изменяется масштаб.
* *random_brightness:* целочисленный диапазон для случайного умножения значений пикселей на.

> **Return:**
* Логическое значение, указывающее, следует ли применять какие-либо искажения.


In [0]:
def should_distort_images(flip_left_right, random_crop, random_scale,
                          random_brightness):

  return (flip_left_right or (random_crop != 0) or (random_scale != 0) or
          (random_brightness != 0))

# **def add_input_distortions()**
Создает операции для применения указанных искажений.
   Во время обучения это может помочь улучшить результаты, если мы запустим изображения
   через простые искажения, такие как зерновые культуры, весы и сальто. Они отражают
   вид изменений, которые мы ожидаем в реальном мире, и поэтому может помочь в обучении
   модель, чтобы справиться с естественными данными более эффективно. Здесь мы берем поставленный
   параметры и построить сеть операций, чтобы применить их к изображению.
   Урожайность
   ~~~~~~~~
   Обрезка выполняется путем размещения ограничивающей рамки в случайном положении в полном объеме.
   образ. Параметр обрезки определяет размер этого поля относительно
   входное изображение. Если это ноль, то поле имеет тот же размер, что и вход, и нет
   обрезка выполняется. Если значение составляет 50%, то поле кадрирования будет вдвое меньше
   ширина и высота ввода. На диаграмме это выглядит так:
  <       width         >
  +---------------------+
  |                     |
  |   width - crop%     |
  |    <      >         |
  |    +------+         |
  |    |      |         |
  |    |      |         |
  |    |      |         |
  |    +------+         |
  |                     |
  |                     |
  +---------------------+
  Scaling
  ~~~~~~~
  Масштабирование очень похоже на обрезку, за исключением того, что ограничивающий прямоугольник всегда
   по центру и его размер изменяется случайным образом в пределах данного диапазона. Например, если
   масштабный процент равен нулю, тогда ограничивающий прямоугольник имеет тот же размер, что и
   вход и масштабирование не применяется. Если это 50%, то ограничительная рамка будет в
   случайный диапазон между половиной ширины и высоты и полным размером.
> **Args:**
* *flip_left_right:* Boolean, следует ли случайным образом отражать изображения по горизонтали.
* *random_crop:* целочисленный процент, задающий общую маржу, используемую вокруг
     ящик для обрезки.
* *random_scale:* целое число в процентах от того, насколько сильно изменяется масштаб.
* *random_brightness:* целочисленный диапазон для случайного умножения значений пикселей на.
     граф.
* *module_spec:* hub.ModuleSpec для используемого модуля изображения.

>**Return:**
     Входной слой jpeg и тензор искаженного результата.


In [0]:

def add_input_distortions(flip_left_right, random_crop, random_scale,
                          random_brightness, module_spec):

  input_height, input_width = hub.get_expected_image_size(module_spec)
  input_depth = hub.get_num_image_channels(module_spec)
  jpeg_data = tf.placeholder(tf.string, name='DistortJPGInput')
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  # Convert from full range of uint8 to range [0,1] of float32.
  decoded_image_as_float = tf.image.convert_image_dtype(decoded_image,
                                                        tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  margin_scale = 1.0 + (random_crop / 100.0)
  resize_scale = 1.0 + (random_scale / 100.0)
  margin_scale_value = tf.constant(margin_scale)
  resize_scale_value = tf.random_uniform(shape=[],
                                         minval=1.0,
                                         maxval=resize_scale)
  scale_value = tf.multiply(margin_scale_value, resize_scale_value)
  precrop_width = tf.multiply(scale_value, input_width)
  precrop_height = tf.multiply(scale_value, input_height)
  precrop_shape = tf.stack([precrop_height, precrop_width])
  precrop_shape_as_int = tf.cast(precrop_shape, dtype=tf.int32)
  precropped_image = tf.image.resize_bilinear(decoded_image_4d,
                                              precrop_shape_as_int)
  precropped_image_3d = tf.squeeze(precropped_image, axis=[0])
  cropped_image = tf.random_crop(precropped_image_3d,
                                 [input_height, input_width, input_depth])
  if flip_left_right:
    flipped_image = tf.image.random_flip_left_right(cropped_image)
  else:
    flipped_image = cropped_image
  brightness_min = 1.0 - (random_brightness / 100.0)
  brightness_max = 1.0 + (random_brightness / 100.0)
  brightness_value = tf.random_uniform(shape=[],
                                       minval=brightness_min,
                                       maxval=brightness_max)
  brightened_image = tf.multiply(flipped_image, brightness_value)
  distort_result = tf.expand_dims(brightened_image, 0, name='DistortResult')
  return jpeg_data, distort_result

In [0]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

# **def add_final_retrain_ops()**
Добавляет новый softmax и полностью связанный слой для обучения и оценки.
  Нам нужно переобучить верхний слой, чтобы определить наши новые классы, поэтому эта функция
  добавляет правильные операции на график, а также некоторые переменные для хранения
  весов, а затем устанавливает все градиенты для обратного прохода.
  Настройка для softmax и полностью связанных слоев основана на:
  https://www.tensorflow.org/tutorials/mnist/beginners/index.html
>**Args:**
* *class_count:* целое число от количества категорий вещей, которые мы пытаемся признать.
* *final_tensor_name:* строка имени для нового конечного узла, который дает результаты.
* *bottleneck_tensor:* вывод основного графа CNN.
* *quantize_layer:* Boolean, указывающий, должен ли новый добавленный слой быть
Инструментарий для квантования с TF-Lite.
* *is_training:* Boolean, указывающий, предназначен ли новый слой для обучения
или Eval.

>**Return:**
* Тензоры для результатов обучения и кросс-энтропии, а также тензоры для
    вход узкого места и ввод правды земли.
> Ссылки
* [tf.truncated_normal](https://www.tensorflow.org/api_docs/python/tf/random/truncated_normal)
* [про квантование в нейронных сетях](https://petewarden.com/2016/05/03/how-to-quantize-neural-networks-with-tensorflow/)
* [tf.contrib.quantize.create_training_graph](https://www.tensorflow.org/api_docs/python/tf/contrib/quantize/create_training_graph)
* [tf.contrib.quantize.create_eval_graph](https://www.tensorflow.org/api_docs/python/tf/contrib/quantize/create_eval_graph)

In [0]:
def add_final_retrain_ops(class_count, final_tensor_name, bottleneck_tensor,
                          quantize_layer, is_training):

  batch_size, bottleneck_tensor_size = bottleneck_tensor.get_shape().as_list()
  assert batch_size is None, 'We want to work with arbitrary batch size.'
  with tf.name_scope('input'):
    bottleneck_input = tf.placeholder_with_default(
        bottleneck_tensor,
        shape=[batch_size, bottleneck_tensor_size],
        name='BottleneckInputPlaceholder')

    ground_truth_input = tf.placeholder(
        tf.int64, [batch_size], name='GroundTruthInput')

  # Организация следующих операций, чтобы их было легче увидеть в TensorBoard.
  layer_name = 'final_retrain_ops'
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      initial_value = tf.truncated_normal(
          [bottleneck_tensor_size, class_count], stddev=0.001)
      layer_weights = tf.Variable(initial_value, name='final_weights')
      variable_summaries(layer_weights)
# tf.truncated_normal Вывод случайных значений из усеченного нормального распределения.
    with tf.name_scope('biases'):
      layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
      variable_summaries(layer_biases)

    with tf.name_scope('Wx_plus_b'):
      logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
      tf.summary.histogram('pre_activations', logits)

  final_tensor = tf.nn.softmax(logits, name=final_tensor_name)

# Функции tf.contrib.quantize переписывают график на месте для
   # квантование. График импортированной модели уже переписан, поэтому после
   # вызывая эти перезаписи, будет только добавленный последний слой
   # преобразился
  if quantize_layer:
    if is_training:
      tf.contrib.quantize.create_training_graph() #Переписывает обучающий input_graph для имитации квантования.
    else:
      tf.contrib.quantize.create_eval_graph()     #Переписывает eval input_graph на месте для симулированного квантования.



  tf.summary.histogram('activations', final_tensor)

  # If this is an eval graph, we don't need to add loss ops or an optimizer.
  # Если это график eval, нам не нужно добавлять операции вычисления ошибки или оптимизатор.
  if not is_training:
    return None, None, bottleneck_input, ground_truth_input, final_tensor

  with tf.name_scope('cross_entropy'):
    cross_entropy_mean = tf.losses.sparse_softmax_cross_entropy(
        labels=ground_truth_input, logits=logits)

  tf.summary.scalar('cross_entropy', cross_entropy_mean)

  with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
    train_step = optimizer.minimize(cross_entropy_mean)

  return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
          final_tensor)


# **add_evaluation_step()**
Вставляет операции, которые нам нужны, чтобы оценить точность наших результатов.
>**Args:**
* *result_tensor: *новый конечный узел, который дает результаты.
* *ground_truth_tensor:* узел, который мы передаем данные истинности землив.

> **Return**:
* Кортеж (шаг оценки, прогноз).

>Ссылки:
* [tf.equal](https://www.tensorflow.org/api_docs/python/tf/math/equal)
  
  

In [0]:
def add_evaluation_step(result_tensor, ground_truth_tensor):

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      prediction = tf.argmax(result_tensor, 1)
      correct_prediction = tf.equal(prediction, ground_truth_tensor) #Возвращает значение истинности (x == y) поэлементно
    with tf.name_scope('accuracy'):
      evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', evaluation_step)
  return evaluation_step, prediction

# **def run_final_eval()**
Запускает окончательную оценку на графике оценки с использованием набора тестовых данных.
> **Args:**
* *train_session: *сессия для графа поездов с тензорами ниже.
* *module_spec: *hub.ModuleSpec для используемого модуля изображения.
* *class_count:* количество классов
* *image_lists:* OrderedDict обучающих изображений для каждой метки.
* *jpeg_data_tensor:* слой для подачи данных изображения jpeg.
* *decoded_image_tensor:* вывод декодирования и изменения размера изображения.
* *resized_image_tensor:* входной узел графа распознавания.
* *bottleneck_tensor:* выходной слой узкого места графа CNN.


In [0]:

def run_final_eval(train_session, module_spec, class_count, image_lists,
                   jpeg_data_tensor, decoded_image_tensor,
                   resized_image_tensor, bottleneck_tensor):

  test_bottlenecks, test_ground_truth, test_filenames = (
      get_random_cached_bottlenecks(train_session, image_lists,
                                    FLAGS.test_batch_size,
                                    'testing', FLAGS.bottleneck_dir,
                                    FLAGS.image_dir, jpeg_data_tensor,
                                    decoded_image_tensor, resized_image_tensor,
                                    bottleneck_tensor, FLAGS.tfhub_module))

  (eval_session, _, bottleneck_input, ground_truth_input, evaluation_step,
   prediction) = build_eval_session(module_spec, class_count)
  test_accuracy, predictions = eval_session.run(
      [evaluation_step, prediction],
      feed_dict={
          bottleneck_input: test_bottlenecks,
          ground_truth_input: test_ground_truth
      })
  tf.logging.info('Final test accuracy = %.1f%% (N=%d)' %
                  (test_accuracy * 100, len(test_bottlenecks)))

  if FLAGS.print_misclassified_test_images:
    tf.logging.info('=== MISCLASSIFIED TEST IMAGES ===')
    for i, test_filename in enumerate(test_filenames):
      if predictions[i] != test_ground_truth[i]:
        tf.logging.info('%70s  %s' % (test_filename,
                                      list(image_lists.keys())[predictions[i]]))

# **build_eval_session()**
Создает восстановленный сеанс eval без операций с поездами для экспорта.
> **Args:**
* *module_spec:*  hub.ModuleSpec для используемого модуля изображения.
* *class_count:* количество классов

> **Return**
* Сессия Eval, содержащая восстановленный граф Eval.
* Вход узкого места, истинность основания, шаг оценки и тензоры прогноза.

  


In [0]:
def build_eval_session(module_spec, class_count):

  # Если квантовать, нам нужно создать правильный eval-граф для экспорта.
  eval_graph, bottleneck_tensor, resized_input_tensor, wants_quantization = (
      create_module_graph(module_spec))

  eval_sess = tf.Session(graph=eval_graph)
  with eval_graph.as_default():
  # Добавить новый слой для экспорта.
    (_, _, bottleneck_input,
     ground_truth_input, final_tensor) = add_final_retrain_ops(
         class_count, FLAGS.final_tensor_name, bottleneck_tensor,
         wants_quantization, is_training=False)

    # Теперь нам нужно восстановить значения из тренировочного графика в граф eval.
    tf.train.Saver().restore(eval_sess, CHECKPOINT_NAME)

    evaluation_step, prediction = add_evaluation_step(final_tensor,
                                                      ground_truth_input)

  return (eval_sess, resized_input_tensor, bottleneck_input, ground_truth_input,
          evaluation_step, prediction)

# **def save_graph_to_file**
Сохраняет график в файл, создавая действительный квантованный в случае необходимости.

In [0]:
def save_graph_to_file(graph_file_name, module_spec, class_count):

  sess, _, _, _, _, _ = build_eval_session(module_spec, class_count)
  graph = sess.graph

  output_graph_def = tf.graph_util.convert_variables_to_constants(
      sess, graph.as_graph_def(), [FLAGS.final_tensor_name])

  with tf.gfile.FastGFile(graph_file_name, 'wb') as f:
    f.write(output_graph_def.SerializeToString())


In [0]:
def prepare_file_system():
  # Set up the directory we'll write summaries to for TensorBoard
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)
  if FLAGS.intermediate_store_frequency > 0:
    ensure_dir_exists(FLAGS.intermediate_output_graphs_dir)
  return

# **def add_jpeg_decoding()**
Добавляет операции, которые выполняют JPEG-декодирование и изменение размера к графику.

> **Args:**
* *module_spec: *hub.ModuleSpec для используемого модуля изображения.

>**Return**
* Тензор для узла для подачи данных JPEG и вывода этапы предварительной обработки.


In [0]:
def add_jpeg_decoding(module_spec):

  input_height, input_width = hub.get_expected_image_size(module_spec)
  input_depth = hub.get_num_image_channels(module_spec)
  jpeg_data = tf.placeholder(tf.string, name='DecodeJPGInput')
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  # Convert from full range of uint8 to range [0,1] of float32.
  decoded_image_as_float = tf.image.convert_image_dtype(decoded_image,
                                                        tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  resize_shape = tf.stack([input_height, input_width])
  resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
  resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                           resize_shape_as_int)
  return jpeg_data, resized_image


# **export_model()**
Экспорт модели для сервировки.
> **Args:**
* *module_spec:* hub.ModuleSpec для используемого модуля изображения.
* *class_count:* количество классов.
* *save_model_dir:* каталог, в котором сохраняются экспортированная модель и переменные.
  

In [0]:
def export_model(module_spec, class_count, saved_model_dir):

# SavedModel должен содержать граф eval.
  sess, in_image, _, _, _, _ = build_eval_session(module_spec, class_count)
  with sess.graph.as_default() as graph:
    tf.saved_model.simple_save(
        sess,
        saved_model_dir,
        inputs={'image': in_image},
        outputs={'prediction': graph.get_tensor_by_name('final_result:0')},
        legacy_init_op=tf.group(tf.tables_initializer(), name='legacy_init_op')
    )

In [0]:



def main(_):
  # Необходимо убедиться, что вывод журнала виден.
  # See https://github.com/tensorflow/tensorflow/issues/3047
  tf.logging.set_verbosity(tf.logging.INFO)

  if not FLAGS.image_dir:
    tf.logging.error('Must set flag --image_dir.')
    return -1

# Подготовьте необходимые каталоги, которые можно использовать во время обучения
  prepare_file_system()

# Посмотрите на структуру папок и создайте списки всех изображений.
  image_lists = create_image_lists(FLAGS.image_dir, FLAGS.testing_percentage,
                                   FLAGS.validation_percentage)
  class_count = len(image_lists.keys())
  if class_count == 0:
    tf.logging.error('No valid folders of images found at ' + FLAGS.image_dir)
    return -1
  if class_count == 1:
    tf.logging.error('Only one valid folder of images found at ' +
                     FLAGS.image_dir +
                     ' - multiple classes are needed for classification.')
    return -1

# Проверьте, означают ли флаги командной строки, что мы применяем какие-либо искажения.
  do_distort_images = should_distort_images(
      FLAGS.flip_left_right, FLAGS.random_crop, FLAGS.random_scale,
      FLAGS.random_brightness)

# Настройте предварительно обученный график.
  module_spec = hub.load_module_spec(FLAGS.tfhub_module)
  graph, bottleneck_tensor, resized_image_tensor, wants_quantization = (
      create_module_graph(module_spec))

# Добавьте новый слой, который мы будем тренировать.
  with graph.as_default():
    (train_step, cross_entropy, bottleneck_input,
     ground_truth_input, final_tensor) = add_final_retrain_ops(
         class_count, FLAGS.final_tensor_name, bottleneck_tensor,
         wants_quantization, is_training=True)

  with tf.Session(graph=graph) as sess:
     # Инициализировать все веса: для модуля их предварительно обученные значения,
     # и для вновь добавленного слоя переподготовки к случайным начальным значениям.
    init = tf.global_variables_initializer()
    sess.run(init)

# Настройка подграфа декодирования изображения.
    jpeg_data_tensor, decoded_image_tensor = add_jpeg_decoding(module_spec)

    if do_distort_images:
      # Мы будем применять искажения, поэтому настройте нужные нам операции.
      (distorted_jpeg_data_tensor,
       distorted_image_tensor) = add_input_distortions(
           FLAGS.flip_left_right, FLAGS.random_crop, FLAGS.random_scale,
           FLAGS.random_brightness, module_spec)
    else:
      # Мы позаботимся о том, чтобы мы вычислили краткие изображения и
       # кэшировал их на диск.
      cache_bottlenecks(sess, image_lists, FLAGS.image_dir,
                        FLAGS.bottleneck_dir, jpeg_data_tensor,
                        decoded_image_tensor, resized_image_tensor,
                        bottleneck_tensor, FLAGS.tfhub_module)

    # Создайте операции, которые нам нужны, чтобы оценить точность нашего нового слоя.
    evaluation_step, _ = add_evaluation_step(final_tensor, ground_truth_input)

    # Объединить все сводки и записать их в файл резюме.
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                         sess.graph)

    validation_writer = tf.summary.FileWriter(
        FLAGS.summaries_dir + '/validation')

     # Создать заставку поезда, которая используется для восстановления значений в граф Eval
     # при экспорте моделей.
    train_saver = tf.train.Saver()

# Запустите обучение столько раз, сколько требуется в командной строке.
    for i in range(FLAGS.how_many_training_steps):
# Получить пакет входных значений узких мест, каждый из которых рассчитывается заново каждый раз
       # время с примененными искажениями или из кеша, хранящегося на диске.
      if do_distort_images:
        (train_bottlenecks,
         train_ground_truth) = get_random_distorted_bottlenecks(
             sess, image_lists, FLAGS.train_batch_size, 'training',
             FLAGS.image_dir, distorted_jpeg_data_tensor,
             distorted_image_tensor, resized_image_tensor, bottleneck_tensor)
      else:
        (train_bottlenecks,
         train_ground_truth, _) = get_random_cached_bottlenecks(
             sess, image_lists, FLAGS.train_batch_size, 'training',
             FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
             decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
             FLAGS.tfhub_module)
# Вставьте узкие места и правду в график и проведите тренинг
       # шаг. Захватите учебные резюме для TensorBoard с опцией `merged`
      train_summary, _ = sess.run(
          [merged, train_step],
          feed_dict={bottleneck_input: train_bottlenecks,
                     ground_truth_input: train_ground_truth})
      train_writer.add_summary(train_summary, i)

# Время от времени распечатывайте, насколько хорошо тренируется график.
      is_last_step = (i + 1 == FLAGS.how_many_training_steps)
      if (i % FLAGS.eval_step_interval) == 0 or is_last_step:
        train_accuracy, cross_entropy_value = sess.run(
            [evaluation_step, cross_entropy],
            feed_dict={bottleneck_input: train_bottlenecks,
                       ground_truth_input: train_ground_truth})
        tf.logging.info('%s: Step %d: Train accuracy = %.1f%%' %
                        (datetime.now(), i, train_accuracy * 100))
        tf.logging.info('%s: Step %d: Cross entropy = %f' %
                        (datetime.now(), i, cross_entropy_value))
# TODO: используйте этот граф eval, чтобы избежать квантования
         # скользящие средние, обновляемые набором проверки, хотя в
         # Практика это делает незначительную разницу.
        validation_bottlenecks, validation_ground_truth, _ = (
            get_random_cached_bottlenecks(
                sess, image_lists, FLAGS.validation_batch_size, 'validation',
                FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
                decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
                FLAGS.tfhub_module))
# Запустите шаг проверки и запишите результаты обучения для TensorBoard
         # с "merged" операцией
        validation_summary, validation_accuracy = sess.run(
            [merged, evaluation_step],
            feed_dict={bottleneck_input: validation_bottlenecks,
                       ground_truth_input: validation_ground_truth})
        validation_writer.add_summary(validation_summary, i)
        tf.logging.info('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
                        (datetime.now(), i, validation_accuracy * 100,
                         len(validation_bottlenecks)))

# Хранить промежуточные результаты
      intermediate_frequency = FLAGS.intermediate_store_frequency

      if (intermediate_frequency > 0 and (i % intermediate_frequency == 0)
          and i > 0):
       # Если мы хотим сделать промежуточное сохранение, сохраните контрольную точку поезда
       # graph, для восстановления в граф eval.
        train_saver.save(sess, CHECKPOINT_NAME)
        intermediate_file_name = (FLAGS.intermediate_output_graphs_dir +
                                  'intermediate_' + str(i) + '.pb')
        tf.logging.info('Save intermediate result to : ' +
                        intermediate_file_name)
        save_graph_to_file(intermediate_file_name, module_spec,
                           class_count)

# По окончании тренировки выполните последнее сохранение контрольной точки поезда.
    train_saver.save(sess, CHECKPOINT_NAME)

# Мы завершили все наши тренинги, поэтому проведите итоговую тестовую оценку
     # некоторые новые изображения, которые мы не использовали раньше.
    run_final_eval(sess, module_spec, class_count, image_lists,
                   jpeg_data_tensor, decoded_image_tensor, resized_image_tensor,
                   bottleneck_tensor)

# Запишите обученный график и метки с весами, сохраненными как
     # константы.
    tf.logging.info('Save final result to : ' + FLAGS.output_graph)
    if wants_quantization:
      tf.logging.info('The model is instrumented for quantization with TF-Lite')
    save_graph_to_file(FLAGS.output_graph, module_spec, class_count)
    with tf.gfile.FastGFile(FLAGS.output_labels, 'w') as f:
      f.write('\n'.join(image_lists.keys()) + '\n')

    if FLAGS.saved_model_dir:
      export_model(module_spec, class_count, FLAGS.saved_model_dir)


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--image_dir',
      type=str,
      default='',
      help='Path to folders of labeled images.'
  )
  parser.add_argument(
      '--output_graph',
      type=str,
      default='/tmp/output_graph.pb',
      help='Where to save the trained graph.'
  )
  parser.add_argument(
      '--intermediate_output_graphs_dir',
      type=str,
      default='/tmp/intermediate_graph/',
      help='Where to save the intermediate graphs.'
  )
  parser.add_argument(
      '--intermediate_store_frequency',
      type=int,
      default=0,
      help="""\
         How many steps to store intermediate graph. If "0" then will not
         store.\
      """
  )
  parser.add_argument(
      '--output_labels',
      type=str,
      default='/tmp/output_labels.txt',
      help='Where to save the trained graph\'s labels.'
  )
  parser.add_argument(
      '--summaries_dir',
      type=str,
      default='/tmp/retrain_logs',
      help='Where to save summary logs for TensorBoard.'
  )
  parser.add_argument(
      '--how_many_training_steps',
      type=int,
      default=4000,
      help='How many training steps to run before ending.'
  )
  parser.add_argument(
      '--learning_rate',
      type=float,
      default=0.01,
      help='How large a learning rate to use when training.'
  )
  parser.add_argument(
      '--testing_percentage',
      type=int,
      default=10,
      help='What percentage of images to use as a test set.'
  )
  parser.add_argument(
      '--validation_percentage',
      type=int,
      default=10,
      help='What percentage of images to use as a validation set.'
  )
  parser.add_argument(
      '--eval_step_interval',
      type=int,
      default=10,
      help='How often to evaluate the training results.'
  )
  parser.add_argument(
      '--train_batch_size',
      type=int,
      default=100,
      help='How many images to train on at a time.'
  )
  parser.add_argument(
      '--test_batch_size',
      type=int,
      default=-1,
      help="""\
     Сколько изображений для тестирования. Этот набор тестов используется только один раз, чтобы оценить
       окончательная точность модели после обучения завершена.
       Значение -1 приводит к использованию всего набора тестов, что приводит к увеличению
       стабильные результаты по пробегам. \
      """
  )
  parser.add_argument(
      '--validation_batch_size',
      type=int,
      default=100,
      help="""\
 Сколько изображений использовать в оценочной партии. Этот набор проверки
       используется гораздо чаще, чем набор тестов, и является ранним индикатором того, как
       Точная модель во время обучения.
       Значение -1 приводит к использованию всего набора проверки, что приводит к
       более стабильные результаты на протяжении итераций обучения, но могут быть медленнее на больших
       тренировочные наборы. \
      """
  )
  parser.add_argument(
      '--print_misclassified_test_images',
      default=False,
      help="""\
      Whether to print out a list of all misclassified test images.\
      """,
      action='store_true'
  )
  parser.add_argument(
      '--bottleneck_dir',
      type=str,
      default='/tmp/bottleneck',
      help='Path to cache bottleneck layer values as files.'
  )
  parser.add_argument(
      '--final_tensor_name',
      type=str,
      default='final_result',
      help="""\
      The name of the output classification layer in the retrained graph.\
      """
  )
  parser.add_argument(
      '--flip_left_right',
      default=False,
      help="""\
      Whether to randomly flip half of the training images horizontally.\
      """,
      action='store_true'
  )
  parser.add_argument(
      '--random_crop',
      type=int,
      default=0,
      help="""\
      A percentage determining how much of a margin to randomly crop off the
      training images.\
      """
  )
  parser.add_argument(
      '--random_scale',
      type=int,
      default=0,
      help="""\
      A percentage determining how much to randomly scale up the size of the
      training images by.\
      """
  )
  parser.add_argument(
      '--random_brightness',
      type=int,
      default=0,
      help="""\
      A percentage determining how much to randomly multiply the training image
      input pixels up or down by.\
      """
  )
  parser.add_argument(
      '--tfhub_module',
      type=str,
      default=(
          'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1'),
      help="""\
      Which TensorFlow Hub module to use.
      See https://github.com/tensorflow/hub/blob/master/docs/modules/image.md
      for some publicly available ones.\
      """)
  parser.add_argument(
      '--saved_model_dir',
      type=str,
      default='',
      help='Where to save the exported graph.')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

ERROR:tensorflow:Must set flag --image_dir.


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
